# Proyecto - 3DCNN

In [91]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import numpy as np
import pandas as pd
import os

In [92]:
class WeightedMultiLabelLogLoss(nn.Module):
    def __init__(self, weight=None):
        super(WeightedMultiLabelLogLoss, self).__init__()
        self.weight = weight

    def forward(self, input, target):
        """
        Computes the weighted multi-label logarithmic loss.

        Args:
            input (torch.Tensor): Predicted probabilities (output of the model).
                                Shape: (batch_size, num_classes)
            target (torch.Tensor): Target labels (ground truth).
                                Shape: (batch_size, num_classes)

        Returns:
            loss (torch.Tensor): Weighted multi-label logarithmic loss.
        """
        epsilon = 1e-15  # Small constant to avoid log(0)

        # Log loss
        log_loss = -target * torch.log(input + epsilon) - (1 - target) * torch.log(1 - input + epsilon)

        # Apply weights if provided
        if self.weight is not None:
            log_loss = log_loss * self.weight

        # Compute mean loss over samples and classes
        loss = log_loss.mean()

        return loss

In [93]:
class Simple3DCNN(nn.Module):
    def __init__(self, num_classes):
        super(Simple3DCNN, self).__init__()
        
        # Capa 3D Conv1
        self.conv1 = nn.Conv3d(in_channels=1, out_channels=32, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool3d(kernel_size=2, stride=2)
        
        # Capa 3D Conv2
        self.conv2 = nn.Conv3d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool3d(kernel_size=2, stride=2)
        
        #Forzar dimensionalidad
        self.fce = nn.Linear(64 * 75, 1)
        
        # Capa completamente conectada
        self.fc1 = nn.Linear(64 * 4 * 4 * 4, 128)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(128, 7)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        
        # forzar dimensionalidad
        x = x.view(-1, 64 * 75)
        x = self.fce(x)
        
        x = x.view(-1, 64 * 4 * 4 * 4)        
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.fc2(x)
        
        return x

# # Crear una instancia del modelo
# num_classes = 7  # Número de clases de salida
# model = Simple3DCNN(num_classes)

# # Imprimir el modelo para ver su estructura
# print(model)


In [94]:
first_object = np.load('volumes/1.2.826.0.1.3680043.1010.npy')

first_object.shape

(256, 256, 300)

In [95]:
#Cargar datos 

class ImageDataGenerator:
    def __init__(self, volumes_ruta,ruta_csv, max_UID):
        self.volumes_ruta = volumes_ruta
        self.max_UID = max_UID
        self.current_UID = 1
        self.names = []
        self.df = pd.read_csv(ruta_csv)
        self.read_volumes_path()
        
    def read_volumes_path(self):
        ruta_carpeta = self.volumes_ruta
        if os.path.exists(ruta_carpeta) and os.path.isdir(ruta_carpeta):
            objetos = os.listdir(ruta_carpeta)
            for objeto in objetos:
                objeto_ruta = os.path.join(ruta_carpeta, objeto)
                self.names.append(objeto_ruta)
                
        
    def get_next_ruta(self):
        next_name = self.names[self.current_UID]
        self.current_UID += 1
        return next_name
        
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.current_UID >= self.max_UID:
            raise StopIteration()
        else:
            # Cargar el volumen
            name = self.get_next_ruta()
            volume = np.load(name)
            #volume.resize((256, 256, 256))
            #label = [float(value) for value in self.df.iloc[self.current_UID][['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7']].tolist()]
            label = self.df.iloc[self.current_UID][['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7']].tolist()
            # Incrementar el UID
            self.current_UID += 1
            
            # Devolver el volumen
            return volume, label

In [96]:
num_classes = 7
model3D = Simple3DCNN(num_classes)

In [97]:
if torch.cuda.is_available():
    print('GPU está disponible')
else:
    print('No se encontró GPU, usando CPU')

GPU está disponible


In [98]:
model3D.to('cuda')

Simple3DCNN(
  (conv1): Conv3d(1, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (relu1): ReLU()
  (pool1): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (relu2): ReLU()
  (pool2): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fce): Linear(in_features=4800, out_features=1, bias=True)
  (fc1): Linear(in_features=4096, out_features=128, bias=True)
  (relu3): ReLU()
  (fc2): Linear(in_features=128, out_features=7, bias=True)
)

In [99]:
data_gen = ImageDataGenerator('volumes', 'train_filtrado.csv', 5)

In [100]:
learning_rate = 1e-3
num_epochs = 1

criterion = WeightedMultiLabelLogLoss()
optimizer = optim.Adam(model3D.parameters(), lr=learning_rate)


model3D.train()

for i, (volume, label) in enumerate(data_gen):
    
    image_tensor = torch.from_numpy(volume).unsqueeze(0).float().to('cuda')
    label_tensor = torch.tensor(label).unsqueeze(0).float().to('cuda')
    
    
    predictions = model3D(image_tensor)
    print("->",predictions)
    print("Shape ->",predictions.shape)
    print("-->",label_tensor)
    loss = criterion(predictions, label_tensor)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    print('Loss: ', loss.item())
    
    

-> tensor([[-0.0024, -0.0595,  0.0606,  0.0322,  0.0265, -0.0399,  0.0384]],
       device='cuda:0', grad_fn=<AddmmBackward0>)
Shape -> torch.Size([1, 7])
--> tensor([[0., 1., 0., 0., 0., 0., 0.]], device='cuda:0')
Loss:  nan
-> tensor([[ 0.3383,  2.3910,  1.3985, -0.3009, -0.6057,  0.2938, -0.2244]],
       device='cuda:0', grad_fn=<AddmmBackward0>)
Shape -> torch.Size([1, 7])
--> tensor([[0., 0., 0., 0., 1., 0., 0.]], device='cuda:0')
Loss:  nan
